<a href="https://colab.research.google.com/github/Numbuh474/cda-5155-tensorflow/blob/master/Copy_of_test_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Clone the entire repo.
%cd /content
%rm -rf "cloned-repo"
!git clone -l -s https://github.com/indera-shsp/cda-5155-tensorflow.git cloned-repo
%cd cloned-repo
!ls

/content
Cloning into 'cloned-repo'...
remote: Enumerating objects: 687, done.
remote: Counting objects: 100% (687/687), done.
remote: Compressing objects: 100% (670/670), done.
remote: Total 687 (delta 18), reused 679 (delta 13), pack-reused 0
Receiving objects: 100% (687/687), 6.67 MiB | 25.78 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/content/cloned-repo
config	  output	   requirements.txt    test_notebook.ipynb
logs	  parse_images.py  sample_output_2.md  train_01
Makefile  README.md	   sample_output.md    utils.py


In [0]:
# from openlocationcode import openlocationcode as olc
# olc.encode(47.365590, 8.524997)
#'8FVC9G8F+6X'

# @see datasets https://github.com/tensorflow/datasets
# @see https://stackoverflow.com/questions/44416764/loading-folders-of-images-in-tensorflow

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pathlib

import time
default_timeit_steps = 1000

# data_dir = '/Volumes/dev/cda-5155-tensorflow/output_256px_grouped_grayscale/set_1'
data_dir = '/content/cloned-repo/output/set_all'
validation_dir = '/content/cloned-repo/output/set_all'

data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))

print('Using tensorflow version: {}'.format(tf.__version__))
print('Using input dir: {} with {} images'.format(data_dir, image_count))


TensorFlow 2.x selected.
Using tensorflow version: 2.0.0
Using input dir: /content/cloned-repo/output/set_all with 582 images


In [0]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') ])
CLASS_NAMES


array(['76XVJMX2+CQFP4', '76XVJMX3+FP9FW', '76XVJMX3+C3P3M',
       '76XVJMX4+2GW5R', '76XVJMX2+9CGRG', '76XVJMX2+FR826',
       '76XVJMX4+W47W5', '76XVJMX4+XQ9HC', '76XVJMX3+CPQHJ',
       '76XVJMX3+9QC65', '76XVMM25+J7M76', '76XVJMW3+JVRWJ',
       '76XVJMX3+C8QQQ', '76XVJMX3+9HXC2', '76XVMM25+63R4V',
       '76XVJMX2+G4Q7X', '76XVJMX3+FJFV5', '76XVJMX3+9GHPW',
       '76XVJMX5+W4424', '76XVJMX3+7JXHG', '76XVJMX4+M47M9',
       '76XVMM25+Q2P34', '76XVMM24+QHV37', '76XVJMX3+877FW',
       '76XVJMX4+M38V4', '76XVMM24+JF4HW', '76XVJMX4+R4V8J',
       '76XVJMX4+R4VJR', '76XVMM24+4G5QJ', '76XVJMX4+WC77W',
       '76XVMM25+923GM', '76XVJMX4+R4VP8', '76XVJMX4+XPHX2',
       '76XVJMX2+CJCWQ', '76XVMM24+PV49G', '76XVMM25+84PJ2',
       '76XVJMX3+C77RF', '76XVJMX2+9Q99G', '76XVJMX4+V4349',
       '76XVJMX2+9VM2V', '76XVJMX4+XG5GP', '76XVJMX2+CPW45',
       '76XVJMX4+R4VWH', '76XVJMX4+WGP67', '76XVMM24+JGX4V',
       '76XVMM24+QJCRV', '76XVJMX4+WHR3R', '76XVMM24+3HRJ8',
       '76XVJMX4+R4VHF',

In [0]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [0]:
image_generator

In [0]:
# Define some parameters for the loader:

epochs = 8

BATCH_SIZE = 128
IMG_HEIGHT = 192
IMG_WIDTH = 256
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
print('STEPS_PER_EPOCH: {}'.format(STEPS_PER_EPOCH))

train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary'
                                                    )

val_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir,
                                                              batch_size=BATCH_SIZE,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              # classes = list(CLASS_NAMES)
                                                              class_mode='binary'
                                                             )

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

STEPS_PER_EPOCH: 5.0
Found 582 images belonging to 58 classes.
Found 582 images belonging to 58 classes.


In [0]:
# image_batch, label_batch = next(train_data_gen)
# show_batch(image_batch, label_batch)

In [0]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))


In [0]:
for f in list_ds.take(5):
  print(f.numpy())

b'/content/cloned-repo/output/set_all/76XVJMX4+WGP67/thumb_0224_lat_29.6498319_lon_-82.3436839.jpg'
b'/content/cloned-repo/output/set_all/76XVJMX4+R4VHF/thumb_0244_lat_29.6496129_lon_-82.344693.jpg'
b'/content/cloned-repo/output/set_all/76XVJMX2+CHXXJ/thumb_0348_lat_29.648623_lon_-82.3485069.jpg'
b'/content/cloned-repo/output/set_all/76XVJMX4+WC77W/thumb_0230_lat_29.649784_lon_-82.343957.jpg'
b'/content/cloned-repo/output/set_all/76XVJMX3+FJFV5/thumb_0517_lat_29.648695_lon_-82.345955.jpg'


In [0]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  # return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
  return img

def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

print('AUTOTUNE: {}'.format(AUTOTUNE))
    

AUTOTUNE: -1


In [0]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)


In [0]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (192, 256, 3)
Label:  [False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False]


In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds


In [0]:
# speedup loading
# train_ds = prepare_for_training(labeled_ds)
# image_batch, label_batch = next(iter(train_ds))
# show_batch(image_batch.numpy(), label_batch.numpy())

In [0]:

def timeit(ds, steps=default_timeit_steps):
  start = time.time()
  it = iter(ds)
  for i in range(steps):
    batch = next(it)
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))


# timeit(train_data_gen)

In [0]:
# timeit(train_ds)

In [0]:

# tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter
# type(labeled_ds.)

# image_paths, labels = labeled_ds # load_base_data(...)
# epoch_size = len(image_paths)
# image_paths = tf.convert_to_tensor(image_paths, dtype=tf.string)
# labels = tf.convert_to_tensor(labels)

# dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

#type(train_ds)

In [0]:
validation_ds, _ = next(train_data_gen)

len(validation_ds)


128

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 192, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 96, 128, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 128, 32)       4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 32, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 49152)             0

In [0]:
len(validation_ds)


128

In [0]:

total_train = 582 # len(train_ds)
total_val =  len(val_data_gen)

print('total_train: {}, total_val: {}'.format(total_train, total_val))

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // BATCH_SIZE,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=1 # total_val // BATCH_SIZE
)

total_train: 582, total_val: 5
Epoch 1/8
4/4 [==============================] - 32s 8s/step - loss: -269.8649 - accuracy: 0.0022 - val_loss: -395.0705 - val_accuracy: 0.0000e+00
Epoch 2/8
4/4 [==============================] - 36s 9s/step - loss: -377.5810 - accuracy: 0.0059 - val_loss: -395.0705 - val_accuracy: 0.0000e+00
Epoch 3/8
4/4 [==============================] - 32s 8s/step - loss: -379.0375 - accuracy: 0.0044 - val_loss: -395.0705 - val_accuracy: 0.0000e+00
Epoch 4/8
4/4 [==============================] - 32s 8s/step - loss: -369.5105 - accuracy: 0.0022 - val_loss: -395.0705 - val_accuracy: 0.0000e+00
Epoch 5/8
4/4 [==============================] - 32s 8s/step - loss: -370.6023 - accuracy: 0.0022 - val_loss: -395.0705 - val_accuracy: 0.0000e+00
Epoch 6/8
4/4 [==============================] - 30s 8s/step - loss: -365.3871 - accuracy: 0.0022 - val_loss: -395.0705 - val_accuracy: 0.0000e+00
Epoch 7/8
4/4 [==============================] - 31s 8s/step - loss: -386.3390 - accura